In [2]:
#1.2?
class WeightedGraph:

    def __init__(self,nodes):
        self.graph=[]
        self.weights={}
        for node in range(nodes):
            self.graph.append([])

    def add_node(self,node):
        self.graph[node]=[]

    def add_edge(self, node1, node2, weight):
        if node2 not in self.graph[node1]:
            self.graph[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def get_weights(self, node1, node2):
        if self.are_connected(node1, node2):
            return self.weights[(node1, node2)]

    def are_connected(self, node1, node2):
        for neighbour in self.graph[node1]:
            if neighbour == node2:
                return True
        return False

    def get_neighbors(self, node):
        return self.graph[node]

    def get_number_of_nodes(self,):
        return len(self.graph)
    
    def get_nodes(self,):
        return [i for i in range(len(self.graph))]

In [165]:
def BellmanFord(g,src,k):
    dist = {}
    path = {}
    num_relax = {}
    
    #initializing distance of all nodes to infinity and dist of 1st node as 0
    for i in range(g.get_number_of_nodes()):
        dist[i] = float("inf")
        path[i] = [src]

        num_relax[i] = 0
        
    dist[src] = 0
    path[src] = [src]

    
    for _ in range(len(g.get_nodes()) - 1):
        difference = False
        for node in g.get_nodes():
            for n in g.get_neighbors(node):
                weight = g.get_weights(node,n)
                if weight != None:
                    if dist[node] + weight < dist[n] and num_relax[node] < k:
                        dist[n] = dist[node] + weight
                        path[n] = path[node] + [n]
                        num_relax[n] += 1
                        difference = True
        if not difference:
            return dist,path


In [166]:
graph = WeightedGraph(5)
graph.add_edge(0, 1, -1)
graph.add_edge(0, 2, 4)
graph.add_edge(1, 2, 3)
graph.add_edge(1, 3, 2)
graph.add_edge(1, 4, 2)
graph.add_edge(3, 2, 5)
graph.add_edge(3, 1, 1)
graph.add_edge(4, 3, -3)

source = 0
k = 3  # Adjusting k according to the constraint

distance, path = BellmanFord(graph, source, k)
print("Test Case 2 - Distance:", distance)
print("Test Case 2 - Path:", path)

graph = WeightedGraph(5)
graph.add_edge(0, 1, 1)
graph.add_edge(1, 2, 1)
graph.add_edge(2, 0, -3)
graph.add_edge(2, 3, 2)
graph.add_edge(3, 4, 2)

source = 0
k = 3  # This is the corrected k value

distance, path = BellmanFord(graph, source, k)
print("Test Case 3 - Distance:", distance)
print("Test Case 3 - Path:", path)

Test Case 2 - Distance: {0: 0, 1: -1, 2: 2, 3: -2, 4: 1}
Test Case 2 - Path: {0: [0], 1: [0, 1], 2: [0, 1, 2], 3: [0, 1, 4, 3], 4: [0, 1, 4]}
Test Case 3 - Distance: {0: -2, 1: -1, 2: 1, 3: 3, 4: 5}
Test Case 3 - Path: {0: [0, 1, 2, 0, 1, 2, 0], 1: [0, 1, 2, 0, 1, 2, 0, 1], 2: [0, 1, 2, 0, 1, 2], 3: [0, 1, 2, 0, 1, 2, 3], 4: [0, 1, 2, 0, 1, 2, 3, 4]}


In [1]:
class BellmanFord_no_k:

    def __init__(self,g,src):
        self.g = g
        self.src = src
        self.dist = {}
        self.path = {}
        
    def startvals(self):
        for i in range(g.get_number_of_nodes()):
            self.dist[i] = float("inf")
            self.path[i] = [self.src]
            
        self.dist[self.src] = 0
        self.path[self.src] = [self.src]
    
    def code(self):
        
        for _ in range(len(self.g.get_nodes()) - 1):
            difference = False
            for node in self.g.get_nodes():
                for n in self.g.get_neighbors(node):
                    weight = self.g.get_weights(node,n)
                    if weight != None:
                        if self.dist[node] + weight < self.dist[n]:
                            self.dist[n] = self.dist[node] + weight
                            self.path[n] = self.path[node] + [n]
                            difference = True
            if not difference:
                return self.dist,self.path
            
    def get_distance(self):
        return self.dist
    
    def get_path(self):
        return self.path


In [2]:
class Dijkstra_no_k:
    def __init__(self, graph):
        self.graph = graph
        self.node_count = graph.get_number_of_nodes()
        self.distances = {node: float('inf') for node in range(self.node_count)}
        self.paths = {node: [] for node in range(self.node_count)}
    
    def find_shortest_paths(self, source):
        self.distances[source] = 0
        self.paths[source] = [source]
        
        initialize_heap = [Item(node, self.distances[node]) for node in range(self.node_count)]
        self.min_heap = MinHeap(initialize_heap)
        
        while not self.min_heap.is_empty():
            current_node = self.min_heap.extract_min()
            node_index = current_node.value
            
            for neighbor in self.graph.get_neighbors(node_index):
                weight = self.graph.get_weights(node_index, neighbor)
                new_distance = self.distances[node_index] + weight
                
                if new_distance < self.distances[neighbor]:
                    self.distances[neighbor] = new_distance
                    self.paths[neighbor] = self.paths[node_index] + [neighbor]
                    self.min_heap.decrease_key(neighbor, new_distance)
    
        return self.distances, self.paths


In [3]:
#2

def neg_wt(g):
    neg = False
    for node in g.get_nodes():
        for n in g.get_neighbors(node):
            wt = g.get_weights(node,n)
            if wt < 0:
                neg =  True
                break
        if neg == True:
            break
    return neg
    

class All_spa:
    
    def __init__(self,g):
        self.g = g
    
    def All_spa(self):
        
        self.dist = {}
        self.path = {}
        
        if neg_wt(self.g) == True:
            for n in self.g.get_nodes():
                bellmang = BellmanFord_no_k(self.g,n)
                bellmang.startvals()
                bellmang.code()
                distn = bellmang.get_distance()
                pathn = bellmang.get_path()
                
                for i in g.get_nodes():
                    self.dist[n][i] = self.distn[i]
                    self.path[n][i] = self.pathn[i][-1]
                    
        else:
            for n in self.g.get_nodes():
                dijkstrag = Dijkstra_no_k(self.g,n)
                distn, pathn = dijkstrag.find_shortest_paths()
                for i in self.g.get_nodes():
                    self.dist[n][i] = distn[i]
                    self.path[n][i] = pathn[i][-1]
                    
                    
    def distances(self,u,v):
        return self.dist[u][v]
    
    def previous(self,u,v):
        return self.path[u][v]

In [ ]:
graph = WeightedGraph(5)
graph.add_edge(0, 1, 1)
graph.add_edge(1, 2, 1)
graph.add_edge(2, 0, -3)
graph.add_edge(2, 3, 2)
graph.add_edge(3, 4, 2)

